In [6]:
%load_ext autoreload
%autoreload 2

from sklearn import datasets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from server.models.ml_models import LinRegModel
from server.models.base_model import MLModel
from sklearn.linear_model import LinearRegression

In [28]:
l = LinearRegression()

In [25]:
a = {'a': 1, 'b': 2}
set(a.keys())

{'a', 'b'}

In [32]:
l = LinRegModel({'n_jobs': -1})

In [34]:
l.fit(X, y)

In [36]:
# l.predict(X)

In [ ]:
l.predict()